In [1]:
import pandas as pd
import numpy as np
import os

# --- 1. 定义文件路径 ---
DATA_DIR = 'data'
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
FINALIZED_DATA_DIR = os.path.join(DATA_DIR, 'finalized')

# 确保最终输出目录存在
os.makedirs(FINALIZED_DATA_DIR, exist_ok=True)

# 各数据源文件路径
### 修正 1: 加载包含目标变量 'target_1d_return' 的正确文件。
price_file = os.path.join(FINALIZED_DATA_DIR, 'data_with_targets.csv') 
tech_file = 'features_technical.csv' # 假设此文件在项目根目录
senti_file = os.path.join(PROCESSED_DATA_DIR, 'sentiment_scores.csv')
funda_file = os.path.join(RAW_DATA_DIR, 'fundamentals.csv')

# --- 2. 加载数据 ---
print("--- 步骤 2: 开始加载数据 ---")

df_price = None
try:
    df_price = pd.read_csv(price_file)
    print(f"✅ 股价与目标数据加载完成，形状: {df_price.shape}")
    # 立即检查目标列是否存在
    if 'target_1d_return' in df_price.columns:
        print("   👍 验证成功: 'target_1d_return' 列已在源文件中找到。")
    else:
        print("   ❌ 严重警告: 'target_1d_return' 列在 'data_with_targets.csv' 中仍然缺失！请检查生成该文件的上一个步骤。")
except FileNotFoundError:
    print(f"❌ 错误：股价与目标数据文件未找到！请检查路径: {price_file}")

df_tech = None
try:
    df_tech = pd.read_csv(tech_file)
    print(f"✅ 技术指标数据加载完成，形状: {df_tech.shape}")
except FileNotFoundError:
    print(f"⚠️ 警告: 技术指标文件未找到于 '{tech_file}'。将跳过此部分。")

df_senti = None
try:
    df_senti = pd.read_csv(senti_file)
    print(f"✅ 情感分数数据加载完成，形状: {df_senti.shape}")
except FileNotFoundError:
    print(f"⚠️ 警告: 情感分数文件未找到于 '{senti_file}'。将跳过此部分。")

df_funda = None
try:
    df_funda = pd.read_csv(funda_file)
    print(f"✅ 基本面数据加载完成，形状: {df_funda.shape}")
except FileNotFoundError:
    print(f"⚠️ 警告: 基本面数据文件未找到于 '{funda_file}'。将跳过此部分。")

### 修正 2: 统一所有数据帧的列名为 'Date' 和 'Ticker' 以匹配后续脚本的要求。
# --- 3. 数据预处理 ---
if df_price is not None:
    print("\n--- 步骤 3: 开始数据预处理 (统一使用 'Date' 和 'Ticker') ---")

    # A. 预处理股价和目标数据 (df_price)
    print("   正在预处理股价和目标数据 (df_price)...")
    # 确保列名为 'Date' 和 'Ticker' (通常 data_with_targets.csv 已经是这个格式)
    if 'date' in df_price.columns:
        df_price.rename(columns={'date': 'Date'}, inplace=True)
    if 'symbol' in df_price.columns:
        df_price.rename(columns={'symbol': 'Ticker'}, inplace=True)
        
    if 'Date' in df_price.columns:
        print("   正在转换 df_price 的 'Date' 列为 datetime 格式...")
        df_price['Date'] = pd.to_datetime(df_price['Date'])
    else:
        print(f"❌ 致命错误: 在 df_price 中找不到 'Date' 或 'date' 列。脚本停止。")
        df_price = None # 设为None以停止后续步骤
        
    # B. 预处理技术指标数据 (df_tech)
    if df_tech is not None:
        print("   正在预处理技术指标数据 (df_tech)...")
        # 统一列名
        df_tech.rename(columns={'date': 'Date', 'symbol': 'Ticker', 'ticker': 'Ticker'}, inplace=True)
        if 'Date' in df_tech.columns:
            print("   正在转换 df_tech 的 'Date' 列为 datetime 格式...")
            df_tech['Date'] = pd.to_datetime(df_tech['Date'])
        else:
            print("   ⚠️ 警告: 在 df_tech 中未找到 'Date' 列。技术指标将无法按日期合并。")

    # C. 预处理情感分数数据 (df_senti)
    if df_senti is not None:
        print("   正在修正情感分数数据 (df_senti)...")
        df_senti.rename(columns={'date': 'Date', 'ticker': 'Ticker', 'symbol': 'Ticker'}, inplace=True)
        if 'Date' in df_senti.columns:
            df_senti['Date'] = pd.to_datetime(df_senti['Date'])
        print("   ✅ df_senti 已准备好进行合并！")

    # D. 预处理基本面数据 (df_funda)
    if df_funda is not None:
        print("   正在准备基本面数据 (df_funda)...")
        df_funda.rename(columns={'Ticker Symbol': 'Ticker', 'symbol': 'Ticker'}, inplace=True)
        print("   ✅ df_funda 已准备好进行合并！")
else:
    print("\n由于基础股价数据加载失败，脚本已停止。")


# --- 4. 合并所有数据 ---
if df_price is not None:
    print("\n--- 步骤 4: 开始合并所有数据 ---")
    
    final_df = df_price.copy()
    print(f"起始数据形状 (股价与目标): {final_df.shape}")

    # 1. 合并技术指标数据
    if df_tech is not None and 'Date' in df_tech.columns and 'Ticker' in df_tech.columns:
        print("   即将合并技术指标...")
        final_df = pd.merge(final_df, df_tech, on=['Date', 'Ticker'], how='left')
        print(f"合并技术指标后形状: {final_df.shape}")
    else:
        print("   跳过合并技术指标数据（数据或必要的 'Date'/'Ticker' 列不可用）。")

    # 2. 合并情感分数数据
    if df_senti is not None and 'Date' in df_senti.columns:
        final_df = pd.merge(final_df, df_senti, on=['Date', 'Ticker'], how='left')
        print(f"合并情感分数后形状: {final_df.shape}")
    else:
        print("   跳过合并情感分数数据（数据或日期列不可用）。")

    # 3. 合并基本面数据
    if df_funda is not None:
        final_df = pd.merge(final_df, df_funda, on='Ticker', how='left')
        print(f"合并基本面数据后形状: {final_df.shape}")
    else:
        print("   跳过合并基本面数据（数据不可用）。")

    print("✅ 所有数据合并完成！")

# --- 5. 检查并保存最终结果 ---
if 'final_df' in locals() and final_df is not None:
    print("\n--- 步骤 5: 检查并保存最终结果 ---")

    print("\n最终合并后 DataFrame 的信息:")
    final_df.info()
    
    print("\n最终 DataFrame 的前5行:")
    print(final_df.head())

    # 检查 'target_1d_return' 是否最终存在
    if 'target_1d_return' in final_df.columns:
        print("\n✅ 最终验证成功: 'target_1d_return' 列存在于最终的数据集中。")
    else:
        print("\n❌ 最终验证失败: 'target_1d_return' 列在合并后丢失！请检查合并逻辑。")

    output_path = os.path.join(FINALIZED_DATA_DIR, 'final_master_dataset.csv')
    final_df.to_csv(output_path, index=False)
    print(f"\n✅ 最终数据集已成功保存到: {output_path}")

--- 步骤 2: 开始加载数据 ---
✅ 股价与目标数据加载完成，形状: (521540, 30)
   👍 验证成功: 'target_1d_return' 列已在源文件中找到。
✅ 技术指标数据加载完成，形状: (521676, 23)
✅ 情感分数数据加载完成，形状: (1211, 5)
✅ 基本面数据加载完成，形状: (102, 188)

--- 步骤 3: 开始数据预处理 (统一使用 'Date' 和 'Ticker') ---
   正在预处理股价和目标数据 (df_price)...
   正在转换 df_price 的 'Date' 列为 datetime 格式...
   正在预处理技术指标数据 (df_tech)...
   正在转换 df_tech 的 'Date' 列为 datetime 格式...
   正在修正情感分数数据 (df_senti)...
   ✅ df_senti 已准备好进行合并！
   正在准备基本面数据 (df_funda)...
   ✅ df_funda 已准备好进行合并！

--- 步骤 4: 开始合并所有数据 ---
起始数据形状 (股价与目标): (521540, 30)
   即将合并技术指标...
合并技术指标后形状: (521540, 51)
合并情感分数后形状: (521540, 54)
合并基本面数据后形状: (521540, 241)
✅ 所有数据合并完成！

--- 步骤 5: 检查并保存最终结果 ---

最终合并后 DataFrame 的信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521540 entries, 0 to 521539
Columns: 241 entries, Date to nameChangeDate
dtypes: bool(6), datetime64[ns](1), float64(150), int64(36), object(48)
memory usage: 938.1+ MB

最终 DataFrame 的前5行:
        Date Ticker    Open_x    High_x     Low_x   Close_x   Volume_x  \
0 2000-03-14 